# 创建交易

## 1.定位指针Point
根据标准交易格式，输入单元指向交易创建者的UTXO，该UTXO对应于区块链中某笔交易的第某个输出单元。由于每笔交易都有唯一的编号，于是定义Simchian中最简单的自定义数据结构，指向UTXO在区块链中的定位指针Pointer。

In [1]:
class Pointer(tuple):
    
    # 初始输入参数为交易编号，UTXO为在交易输出中的索引
    def __new__(cls,tx_id,n):
        # 初始化参数为tx_id, n
        return super(Pointer,cls).__new__(cls,(tx_id,n))
    
    # 交易编号
    @property
    def tx_id(self):
        return self[0]
    
    # 输出单元索引
    @property
    def n(self):
        return self[1]
    
    def __repr__(self):
        return "Pointer(tx_id:{0},n:{1})".format(self[0],self[1])

In [2]:
from simchain import Network, Pointer, sha256d

In [3]:
# 创建一个定位指针
p = Pointer(1, 2)
p

Pointer(tx_id:1,n:2)

In [4]:
# 创建一个P2P网络
net = Network()
# 将0号节点命名为张三
zhangsan = net.peers[0]
# 张三只有一个UTXO
len(zhangsan.get_utxo())

2019-05-17 14:16:55,225 - A blockchain p2p network created,12 peers joined
2019-05-17 14:16:55,241 - genesis block has been generated


1

In [5]:
# 获取张三的UTXO
utxo = zhangsan.get_utxo()[0]
# 获取张三UTXO的定位指针
utxo.pointer

Pointer(tx_id:1816e0f3f29092910fc879cd7bf0618273a39e0456dfda1636e87434f58ef14a,n:0)

In [6]:
# 获取张三的UTXO金额
utxo.vout.value

100000

In [7]:
# 获取创世区块中的交易
txs = zhangsan.blockchain[0].txs
txs

[Tx(id:1816e0f3f29092910fc879cd7bf0618273a39e0456dfda1636e87434f58ef14a)]

In [8]:
# 获取交易的第一个输出单元
txs[0].tx_out[0]

Vout(to_addr:1HYCEuvH5g8j4siKtYqVwov6xaSFnLgXD9,value:100000)

In [9]:
# 指向张三的地址
zhangsan.addr

'1HYCEuvH5g8j4siKtYqVwov6xaSFnLgXD9'

## 2.输入单元Vin
交易输入和输出的数量是不固定的，是输入单元和输出单元的集合。定义输入单元和输出单元数据结构。

In [10]:
class Vin(tuple):
    
    # 初始输入参数，定位指针、公钥和签名
    def __new__(cls,to_spend,signature,pubkey):
        return super(cls,Vin).__new__(cls,(to_spend,signature,pubkey))
        
    # 输入单元使用的UTXO在区块链中的定位
    @property
    def to_spend(self):
        return self[0]
    
    # 数字签名
    @property
    def signature(self):
        return self[1]
    
    # 公钥
    @property
    def pubkey(self):
        return self[2]

    # 签名脚本
    @property
    def sig_script(self):
        return self[1]+self[2]

    def __repr__(self):
        return "Vin(to_spend:{0}, signature:{1}, pubkey:{2})".format(self[0],self[1],self[2])

In [11]:
from simchain import Vin
# 创建一个输入单元
vin = Vin(1, 2, 3)
vin

Vin(to_spend:1,signature:2,pubkey:3)

In [12]:
# 将4号节点命名为李四
lisi = net.peers[4]
# 张三给李四转账100
zhangsan.create_transaction(lisi.wallet.addrs[-1], 100)
# 获取张三创建的当前交易
tx = zhangsan.txs[0]
# 获取交易的第一个输出单元Vin
tx.tx_in[0]

2019-05-17 14:16:56,131 - peer(6, 3)(pid=0) created a transaction


Vin(to_spend:Pointer(tx_id:1816e0f3f29092910fc879cd7bf0618273a39e0456dfda1636e87434f58ef14a,n:0),signature:b'\x10\xf4y!\xf6\xe9\xe4u\xec\xd9X\xf1\xab,\xd0b\xb7\x1cI\xb8\xe6\xe0nf\x8a\x9a\xdb`\x0c\x070\x05Ja!,\xcam)\x9d\xe1\xcf\xb1d\xa6\xe6\xc4j\x86 \x98ZX)*D}~\x18^\xa1&0\xfa',pubkey:b'\xf7\x98\x12\x8d\xc2W\x07\xa1l!,\xa2\x8c~5\xb9Z\xf8\x99\xb3\x97b\xeb;\x1e\x9e\xaf\xed\xda\xfaREW\xd2\xd6\x1f\x02=\xc2\x9f[\xb8J\x89\xd40\xfa;\xd6\xf1\x93\x14y\xee/\xce\xcc\x15\x9bH\xcd\x8cf\x83')

## 3.输出单元Vout

In [13]:
class Vout(tuple):
    
    def __new__(cls,to_addr,value):
        return super(Vout,cls).__new__(cls,(to_addr,value))
        
    # 交易接收者的地址    
    @property
    def to_addr(self):
        return self[0]
    
    # 转账金额
    @property
    def value(self):
        return self[1]

    # 公钥脚本
    @property
    def pubkey_script(self):
        script = "OP_DUP OP_ADDR {0} OP_EQ OP_CHECKSIG".format(self[0])
        return script

    def __repr__(self):
        return "Vout(to_addr:{0},value:{1})".format(self[0],self[1])

In [14]:
from simchain import Vout
vout = Vout(11, 100)
vout

Vout(to_addr:11,value:100)

In [15]:
# 交易输出的第1个输出单元
tx.tx_out[0]

Vout(to_addr:1NYvthL4HG9crMXJGCFjKGCT7xCQQWVZNr,value:100)

In [16]:
# 第2个交易输出单元，指向张三自己地址的UTXO，10分的交易费
tx.tx_out[1]

Vout(to_addr:1HYCEuvH5g8j4siKtYqVwov6xaSFnLgXD9,value:99890)

## 4.交易Tx
交易由输入和输出组成，将其定义为Tx。

In [17]:
class Tx(tuple):
    
    # 输入初始参数
    def __new__(cls,tx_in,tx_out,fee=0,timestamp=0,nlocktime=0):
        return super(Tx,cls).__new__(cls,(tx_in,
                                          tx_out,
                                          fee,
                                          timestamp,
                                          nlocktime))
    
    # 交易输入
    @property
    def tx_in(self):
        return self[0]
    
    # 交易输出
    @property
    def tx_out(self):
        return self[1]
    
    # 交易费
    @property
    def fee(self):
        return self[2]
    
    # 交易锁定时间
    @property
    def nlocktime(self):
        return self[3]
    
    # 是否为创币交易
    @property
    def is_coinbase(self) -> bool:
        return len(self[0]) == 1 and self[0][0].to_spend is None

    # 定义类方法，返回一个创币交易，创币交易只有一个Vin和Vout
    # Vin的定位指针为空，签名是一个32字节的系统随机数，Vout指向获胜节点的地址
    @classmethod
    def create_coinbase(cls, pay_to_addr, value):
        import os
        return cls(
            tx_in = [Vin(to_spend=None,
                         signature= str(os.urandom(32)),
                         pubkey=None)],
            tx_out = [Vout(to_addr=pay_to_addr,
                           value=value)]
            )

    # 交易编号，调用eec.py模块中的sha256d双哈希函数，对交易数据进行哈希运算
    @property
    def id(self):
        return sha256d(self.to_string())
    
    def to_string(self):
        return "{0}{1}{2}".format(self[0],
                                  self[1],
                                  self[3])

    def __repr__(self):
        return "Tx(id:{0})".format(self.id)

In [18]:
from simchain import Pointer, Vin, Vout, Tx
# 创建一个Pointer
pointer = Pointer(2, 3)
# 创建一个Vin
vin = Vin(pointer, 2, 3)
# 创建一个Vout
vout = Vout(2, 4)
# 创建交易
tx = Tx([vin], [vout])
tx

Tx(id:aadd7b0c9ed66235a33514ea8ac2c7be35deba97d8c2b7e801384c9ec0b95c9c)

如果修改定位指针Pointer，则交易的编号会发生变化。

In [19]:
pointer1 = Pointer(3, 4)
vin1 = Vin(pointer1, 2, 3)
tx.tx_in[0] = vin1
tx

Tx(id:f65798a2e3081198775ad6a424df37e356e70cce7468e19e6434bf95a9269f93)

直接调用类方法create_coinase()创建一个创币交易

In [20]:
# 输入获胜矿工地址和金额
base = Tx.create_coinbase(1, 20)
base

Tx(id:1829c7e4c6eeb0cde88049f9555f4b955b9d50078d66c27f2c76da018a176752)

## 5.UTXO和UTXO集
建立单独的微型交易数据库，仅存储和自己有关的交易。每个节点将自己区块链中所有的UTXO独立存储在一个数据集中，称为UTXO集，用UTXO_SET表示，将微型交易数据库中指向自己地址的输出单元封装成UTXO，用OWN_UTXO_SET表示。当节点需要创建交易时，从OWN_UTXO_SET中查找，但当节点需要验证某笔交易使用的UTXO是否有效时，只能从UTXO_SET中查找。

In [21]:
class UTXO(tuple):
    
    # 输入长寿为输出单元Vout，Vout在区块链中的定位指针Pointer，是否来自创币交易以及是否被消费和确认，两种状态有默认值
    def __new__(cls,
                vout,
                pointer,
                is_coinbase,
                unspent=True,
                confirmed=False,
                ):
        return super(UTXO,cls).__new__(cls,(vout,
                                            pointer,
                                            is_coinbase,
                                            unspent,
                                            confirmed))
        
    # 该UTXO封装的输出单元
    @property
    def vout(self):
        return self[0]
    
    # 输出单元在区块链中的定位
    @property
    def pointer(self):
        return self[1]
    
    # 封装的输出单元是否来自创币交易
    @property
    def is_coinbase(self):
        return self[2]

    # 公钥脚本，与输出单元的公钥脚本一致
    @property
    def pubkey_script(self):
        return self[0].pubkey_script

    # 是否被消费
    @property
    def unspent(self):
        return self[3]

    # 是否被确认
    @property
    def confirmed(self):
        return self[4]

    # 返回新的UTXO，修改原UTXO的两个状态
    def _replace(self,unspent = True, confirmed = False):
        return UTXO(self[0],self[1],self[2],unspent,confirmed)
    
    def __repr__(self):
        return "UTXO(vout:{0},pointer:{1})".format(self[0],self[1])

In [22]:
# 创建一个定位指针
pointer = Pointer(3, 4)
# 创建一个输出单元
vout = Vout(2, 100)

In [23]:
from simchain import UTXO
# 创建一个UTXO对象
utxo = UTXO(vout, pointer, False)
utxo

UTXO(vout:Vout(to_addr:2,value:100),pointer:Pointer(tx_id:3,n:4))

In [24]:
# 创建一个新的UTXO对象，替换原UTXO的状态
utxo1 = utxo._replace(False, True)
utxo1.unspent, utxo1.confirmed

(False, True)

In [25]:
# 原UTXO状态并没有改变
utxo.unspent, utxo.confirmed

(True, False)

In [26]:
# 网络中添加新节点
net.add_peer()
# 将新节点命名为王五
wangwu = net.peers[-1]
# 张三给王五转账
zhangsan.create_transaction(wangwu.wallet.addrs[-1], 100)

2019-05-17 14:16:58,236 - A new peer joined in --> peer(75, 77)(pid=12)
2019-05-17 14:16:58,282 - peer(6, 3)(pid=0) created a transaction


True

In [27]:
# 张三将交易广播到网络中
zhangsan.broadcast_transaction()

2019-05-17 14:16:58,415 - peer(6, 3)(pid=0) sent a transaction to network
2019-05-17 14:16:59,361 - peer(6, 3)(pid=0)'s transaction verified by 12 peers


12

In [28]:
# 获取张三未确认的UTXO
zhangsan.get_unconfirmed_utxo()

[UTXO(vout:Vout(to_addr:1HYCEuvH5g8j4siKtYqVwov6xaSFnLgXD9,value:99890),pointer:Pointer(tx_id:55e190ede2154d5c612f0c3e221354f37bacd4623341af50c2df4cdfdb9a4ec5,n:1))]

In [29]:
# 获取王五未确认的UTXO，来自同一条交易
wangwu_utxo = wangwu.get_unconfirmed_utxo()
wangwu_utxo

[UTXO(vout:Vout(to_addr:1FaAoyp2KikHamJhKRHufUeHnPzewGXDBH,value:100),pointer:Pointer(tx_id:55e190ede2154d5c612f0c3e221354f37bacd4623341af50c2df4cdfdb9a4ec5,n:0))]

In [30]:
# 达成共识
net.consensus()

2019-05-17 14:16:59,559 - 6 peers are mining
2019-05-17 14:17:09,526 - peer(33, 46)(pid=10) is winner,9.966892004013062 secs used
2019-05-17 14:17:10,513 - Block(hash:000017a1b83a7a51a03fbe5c48683a2d8580ae79e69731bb38480a8b228a3ef2) received by 12 peers


In [31]:
# 获取张三未确认的UTXO
zhangsan.get_unconfirmed_utxo()

[]

In [32]:
# 获取王五未确认的UTXO
wangwu.get_unconfirmed_utxo()

[]

Simchain中的UTXO_SET使用字典存储，元素key为UTXO对象的定位指针Pointer，这样可以方便地根据定位指针来查找UTXO。每个完整节点都有全网UTXO_SET备份，和区块链数据同步的。

In [33]:
pointer = Pointer(wangwu_utxo[0].pointer.tx_id, 0)
wangwu.utxo_set[pointer]

UTXO(vout:Vout(to_addr:1FaAoyp2KikHamJhKRHufUeHnPzewGXDBH,value:100),pointer:Pointer(tx_id:55e190ede2154d5c612f0c3e221354f37bacd4623341af50c2df4cdfdb9a4ec5,n:0))

In [34]:
type(wangwu.utxo_set)

dict

## 一般交易的创建

In [35]:
# 输入参数为创建交易节点、支付地址和支付金额
def create_normal_tx(peer,to_addr,value) :
    # 获取节点自己的UTXO列表和余额
    utxos,balance = peer.get_utxo(), peer.get_balance()
    
    # 获取节点的交易费和钱包
    fee,wallet = peer.fee,peer.wallet
    
    # 初始化交易输入和输出列表
    tx_in,tx_out = [],[]
    
    # 计算总支付金额，数量为应支付金额加交易费
    value = value + fee
    
    # 判断余额是否足够支付
    if balance  < value:
        logger.info('no enough money for transaction for {0}(pid = {1})'.format(peer,peer.pid))
        return
        
    # 初始化需要使用的UTXO数量n，以及金额之和    
    need_to_spend,n = 0,0
    
    # 在自己的UTXO列表中找到前n个UTXO
    for i,utxo in enumerate(utxos):
        need_to_spend += utxo.vout.value
        if need_to_spend >= value:
            n = i+1
            break
            
    # 如果前n个UTXO金额之和大于应支付总金额        
    if need_to_spend > value:
        # 获取自己的最新地址
        my_addr = wallet.addrs[-1]
        # 交易输出单元为两条，分别指向接收方的地址和指向自己的地址
        tx_out +=[Vout(to_addr,value-fee),Vout(my_addr,need_to_spend-value)]
    
    # 如果前n个UTXO金额之和等于应支付总金额    
    else:
        # 只有一条指向接收方地址的输出单元
        tx_out += [Vout(to_addr,value-fee)]
            
    # 使用前n个UTXO创建交易的n个输入单元        
    for utxo in utxos[:n]:
        
        # 获取当前UTXO指向的地址
        addr = utxo.vout.to_addr
        
        # 在钱包中找到该地址的索引
        idx = wallet.addrs.index(addr)
        
        # 找到该地址对应的私钥和公钥
        sk,pk = wallet.keys[idx].sk,wallet.keys[idx].pk
        
        # 选择签名明文，为UTXO定位指针，公钥和输出单元
        string = str(utxo.pointer) + str(pk.to_bytes()) + str(tx_out)
        
        # 计算明文的哈希值
        message = build_message(string)
        
        # 使用私钥对明文签名
        signature = sk.sign(message)
        
        # 创建单个输入单元并添加到交易输入列表
        tx_in.append(Vin(utxo.pointer,signature,pk.to_bytes()))
        
    return tx_in,tx_out,fee
    
# 计算明文的哈希值，调用sha256d双哈希函数    
def build_message(string):
    return sha256d(string).encode()

In [36]:
from simchain import Network, Tx
from simchain.peer import create_normal_tx
net = Network()
zhangsan, lisi = net.peers[0], net.peers[1]
# 张三创建一笔交易
results = create_normal_tx(zhangsan, lisi.addr, 100)
# 返回交易的输入列表
results[0]

2019-05-17 14:17:11,500 - A blockchain p2p network created,12 peers joined
2019-05-17 14:17:11,500 - genesis block has been generated


[Vin(to_spend:Pointer(tx_id:48cd191d5d596a4a65f6d90cd37448ea97739fa2dc37fed0f934826f2f3db680,n:0),signature:b'\xc7\x0cF\xb6B\x8a\xc8\x98%`6\xf8\xc8\x1c\xe59\xea\x08\x93F\xa4\xd2\x00\x9e~m\x94B\x97\xbc\x9bhf\xe1PHq"\xddC2<\xf2\x1f\x91\x02[\x8b\xfb\x03\xee\xc0k\x14^\x01\x7f\xcbP\xa6\xcf8\xf7\x9e',pubkey:b'\x14\x1c\xbf\x97\xf8\x82\xb6\xb0\xbamx\xfagT\x07\xd2m\xb1j\xe4"z\xca8\x06\xce\x01\x84v\xaa\xd0\x99\x95\xcc\x08\xd1k\xa0\xd47\x9a~?\x89\xf4\xf5\xbc\xdf~tJ$\x0e\xc8<\xb8\xac\xe9\x80i\xc8R\xa7\x18')]

In [37]:
# 返回交易的输出列表
results[1]

[Vout(to_addr:1Kia3uVCvPyJN8C41fMhuXywxw66KtspPB,value:100),
 Vout(to_addr:14wDYis6r8Wq2kPwkXnVa5kWomonimEvCV,value:99890)]

In [38]:
# 返回交易费
results[2]

10

In [39]:
# 通过输入、输出、交易费创建交易
tx = Tx(results[0], results[1], results[2])
tx

Tx(id:0cb147eb5237888328bbe359d8f47b7dcaaa81e7fe8827b0dca1b65f200903aa)

交易创建好后，在广播交易之前，先将交易放进自己的交易池中，然后标记交易。

In [40]:
# 输入参数为UTXO_SET和创建的交易
def sign_utxo_from_tx(utxo_set,tx):
    
    # 遍历交易所有的输入单元Vin
    for vin in tx.tx_in:
        
        # 获取Vin的定位指针
        pointer = vin.to_spend
        
        # 在UTXO_SET中找到该定位指针对应的UTXO
        utxo = utxo_set[pointer]
        
        # 创建一个新的UTXO，与原UTXO相比，仅仅是修改了消费状态
        utxo = utxo._replace(unspent = False)
        
        # 用新的UTXO替换之前的UTXO
        utxo_set[pointer] = utxo

In [41]:
from simchain.peer import sign_utxo_from_tx
# 对交易使用过的UTXO进行标记
sign_utxo_from_tx(zhangsan.utxo_set, tx)
# 张三的UTXO为空
zhangsan.get_utxo()

[]

In [42]:
# 张三的未确认UTXO也为空
zhangsan.get_unconfirmed_utxo()

[]

如果全网允许从未确认的有效交易中获取UTXO，则节点还需要将该笔交易的每个输出单元Vout封装为UTXO，加入到UTXO_SET中，但这些UTXO的状态应标记为未确认。

In [43]:
# 输入为节点和创建的交易
def add_tx_to_mem_pool(peer,tx):
    
    # 将交易添加到交易池中
    peer.mem_pool[tx.id] = tx
    # 如果允许从未确认的有效交易中获取UTXO
    if peer.allow_utxo_from_pool:
        add_utxos_from_tx_to_set(peer.utxo_set,tx)

# 将封装好的UTXO添加到UTXO_SET中 
def add_utxos_from_tx_to_set(utxo_set,tx):
    utxos = find_utxos_from_tx(tx)
    for utxo in utxos:
        utxo_set[utxo.pointer] = utxo

# 将交易中所有的Vout封装成UTXO，默认状态为未消费，未确认
def find_utxos_from_tx(tx):
    return [UTXO(vout,Pointer(tx.id,i),tx.is_coinbase)
            for i,vout in enumerate(tx.tx_out)]

In [44]:
from simchain.peer import add_tx_to_mem_pool
add_tx_to_mem_pool(zhangsan, tx)
# 张三的未确认UTXO
zhangsan.get_unconfirmed_utxo()

[UTXO(vout:Vout(to_addr:14wDYis6r8Wq2kPwkXnVa5kWomonimEvCV,value:99890),pointer:Pointer(tx_id:0cb147eb5237888328bbe359d8f47b7dcaaa81e7fe8827b0dca1b65f200903aa,n:1))]

In [45]:
# 张三的交易池中有一条交易
zhangsan.mem_pool

{'0cb147eb5237888328bbe359d8f47b7dcaaa81e7fe8827b0dca1b65f200903aa': Tx(id:0cb147eb5237888328bbe359d8f47b7dcaaa81e7fe8827b0dca1b65f200903aa)}

In [46]:
# 李四的交易池为空
lisi.mem_pool

{}

In [47]:
# 李四的余额没变
lisi.get_balance()

100000

In [48]:
# 李四没有未确认的UTXO，因为张三并没有广播交易
lisi.get_unconfirmed_utxo()

[]

In [49]:
# 李四验证交易有效
lisi.verify_transaction(tx, lisi.mem_pool)

True

In [50]:
# 将交易添加到李四的交易池
add_tx_to_mem_pool(lisi, tx)
# 李四账户余额增加
lisi.get_balance()

100100

In [51]:
# 有一个未确认UTXO
lisi.get_unconfirmed_utxo()

[UTXO(vout:Vout(to_addr:1Kia3uVCvPyJN8C41fMhuXywxw66KtspPB,value:100),pointer:Pointer(tx_id:0cb147eb5237888328bbe359d8f47b7dcaaa81e7fe8827b0dca1b65f200903aa,n:0))]

## 创币交易的创建
创币交易不需要花费UTXO，而且只有指向获胜共识节点地址的唯一输出单元，金额为共识奖励和交易费之和，Simchain中默认共识奖励为500分，每笔交易费为10分。

In [52]:
# 输出参数为获胜节点地址和金额
def create_coinbase(cls, pay_to_addr, value):
    return cls(
        tx_in = [Vin(to_spend=None,
                     signature= str(os.urandom(32)),
                     pubkey=None)],
        tx_out = [Vout(to_addr=pay_to_addr,
                       value=value)]
        )

# 输入参数为区块交易列表
def calculate_fees(self,txs=[]):
    return sum(tx.fee for tx in txs)

In [53]:
# 给李四发放奖励
tx.create_coinbase(lisi.addr, 510)

Tx(id:12e723205568781379eb0328da38477a745dade7c5e67e623f7d9800291a3ae4)

In [54]:
# 李四计算交易费
lisi.calculate_fees([tx])

10

在创建交易的过程中，无论用户使用的UTXO_SET还是OWN_UTXO_SEt，都不需要和其他任何节点进行数据交换，该过程完全是可以离线进行的。